In [2]:
words = open('../names.txt','r').read().splitlines()

In [11]:
import torch

In [8]:
# stoi and itos dicts
chars = ['.']+sorted([chr(char) for char in range(ord('a'),ord('z')+1)])+['<s>']
stoi = {s:i for i,s in enumerate(chars)}
itos = {i:s for s,i in stoi.items()}

In [19]:
N = torch.zeros(28,28,27) # cuz I wouldn't expect <s> for the 3rd dimension. and anyway its just a filler character.

In [20]:
#filling the N array.
for word in words:
    chs = ['.']+['<s>']+list(word)+['.']
    for cha,chb,chc in zip(chs,chs[1:],chs[2:]):
        # print(f'cha,chb,chc : {cha,chb,chc}')
        i,j,k = stoi[cha],stoi[chb],stoi[chc]
        N[i,j,k]+=1

In [ ]:
N.sum() # the count will be same as you can imagine each increment is like sliding over the name and turns out with your current approach you are sliding as much as when you were doing bigrams with just window size of 3.


In [110]:
P = (N+1).float()
# print(P.sum(dim=2).shape) #wrong # torch.Size([28, 28])
# print(P.shape) # torch.Size([28, 28, 27])
# P.sum(dim=2,keepdim=True).shape #torch.Size([28, 28, 1])
P/=P.sum(dim=2,keepdim=True)

In [111]:
log_likelihood=0.0
ncount=0
for w in words:
    # chs = ['.']+['<s>']+list(word)+['.']
    chs = ['.']+['<s>']+list(w)+['.'] #bug identified.
    for cha,chb,chc in zip(chs,chs[1:],chs[2:]):
        i,j,k = stoi[cha],stoi[chb],stoi[chc]
        prob = P[i,j,k]
        log_likelihood+=torch.log(prob)
        ncount+=1
neg_log_likelihood = -log_likelihood
mean_neg_log_likelihood = neg_log_likelihood/ncount

In [ ]:
print(f'{neg_log_likelihood=}')
print(f'{mean_neg_log_likelihood=}')
print(ncount)

In [ ]:
mean_neg_log_likelihood

In [ ]:
g = torch.Generator().manual_seed(2147483647)


for _ in range(10):
    nameList = []
    ix,iy =0,27
    while True:
        p = P[ix,iy]
        iz = torch.multinomial(p,num_samples=1,replacement=True,generator=g).item()
        if iz==0:
            break
        nameList.append(itos[iz])
        ix,iy=iy,iz
    print(''.join(nameList))

Implementing the neural net.

In [4]:
nnNames = list(words)

In [5]:
import random
random.shuffle(nnNames)

train_size = int(0.8 * len(nnNames))
dev_size = int(0.1 * len(nnNames))
test_size = len(nnNames) - train_size - dev_size

train_data = nnNames[:train_size]
dev_data = nnNames[train_size:train_size + dev_size]
test_data = nnNames[train_size + dev_size:]

In [162]:
xs,ys,zs = [],[],[]

# for name in nnNames: # should be using train data here
for name in train_data:
    # n = ['.']+['<s>']+list(word)+['.'] #same bug using word instead of name...
    n = ['.']+['<s>']+list(name)+['.']
    for cha,chb,chc in zip(n,n[1:],n[2:]):
        xs.append(stoi[cha])
        ys.append(stoi[chb])
        zs.append(stoi[chc])

xs,ys,zs = torch.tensor(xs),torch.tensor(ys),torch.tensor(zs)

In [ ]:
xs.shape,ys.shape,zs.shape

In [10]:
import torch.nn.functional as F

In [121]:
# was thinking like how we used W as the same dimension as N in bigram I think I should do the same here
W = torch.randn((28,28,27),requires_grad=True)


In [122]:
cpyW = W.clone().detach()

Backpropagation

In [ ]:
zenc = F.one_hot(zs,num_classes=27).float()
zenc.shape

In [ ]:
W[1,1,:]

In [ ]:
W[[1,2],[1,2]]

In [ ]:
step_size = 0.01*5000
for i in range(10):
    #forward pass
    logits  =W[xs,ys,:]
    loss = F.cross_entropy(logits,zenc)+ 0*(W**2).mean() #p_loss + smoothing factor

    #backward pass
    W.grad = None
    loss.backward()

    #update 
    W.data+=W.grad*step_size*-1

    print(loss)

Code to get the loss closer to the theoretical limit (one obtained by the counting method)

Backpropagation to get the loss value of the nn method as close as counting method.

In [ ]:
step_size = 0.01*1
d_loss = torch.tensor(2.2120)
for i in range(10):
    #forward pass
    logits  =W[xs,ys,:]
    p_loss = F.cross_entropy(logits,zenc)+ 0*(W**2).mean() #p_loss + smoothing factor
    loss = torch.abs(p_loss-d_loss)

    #backward pass
    W.grad = None
    loss.backward()

    #update 
    W.data+=W.grad*step_size*-1

    print(loss)
print(p_loss)

In [ ]:
#sampling
g = torch.Generator().manual_seed(2147483647)

for i in range(10):
    out = []
    ix,iy= 0,27 # ix = . iy = <s>
    while True:
        l_logits = W[ix,iy] # gives the 3rd dimension of the weight corresponding to each 27 neurons of the pair ix,iy
        l_counts = l_logits.exp()
        l_p = l_counts/l_counts.sum()

        iz = torch.multinomial(l_p,num_samples=1,replacement=True,generator=g).item()
        if iz==0:
            break
        out.append(itos[iz])
        ix,iy = iy,iz
    print(''.join(out))

Okay so its a little better now.

Turns out ther was a bug in the loop that prepared the xs,ys and zs.

Testing of the model on the test set.

In [321]:
tx,ty,tz = [],[],[]
for name in test_data:
    n = ['.']+['<s>']+list(name)+['.']
    for cha,chb,chc in zip(n,n[1:],n[2:]):
        tx.append(stoi[cha])
        ty.append(stoi[chb])
        tz.append(stoi[chc])

tx,ty,tz = torch.tensor(tx),torch.tensor(ty),torch.tensor(tz)
tzenc = F.one_hot(tz,num_classes=27).float()

In [ ]:
tx.shape,ty.shape,tz.shape,tzenc.shape

Evaluating loss

In [ ]:
t_logits = W[tx,ty]
loss = F.cross_entropy(t_logits,tzenc)
loss

Saving the model

In [14]:
# torch.save(W, 'trigram_nn_model.pth') dont call it unnecessarily it can override things.
W = torch.load('trigram_nn_model.pth')

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_23756\3941473048.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  W = torch.load('trigram_nn_model.pth')


Implementing the model smoothing

In [13]:
dx,dy,dz = [],[],[]
for name in dev_data:
    n = ['.']+['<s>']+list(name)+['.']
    for cha,chb,chc in zip(n,n[1:],n[2:]):
        dx.append(stoi[cha])
        dy.append(stoi[chb])
        dz.append(stoi[chc])
    
dx,dy,dz = torch.tensor(dx),torch.tensor(dy),torch.tensor(dz)
dzenc = F.one_hot(dz,num_classes=27).float()
dx.shape,dy.shape,dz.shape,dzenc.shape

(torch.Size([22879]),
 torch.Size([22879]),
 torch.Size([22879]),
 torch.Size([22879, 27]))

In [378]:
step_size = 0.01*5
smoothing_factor=0
for i in range(10):
    #forward pass
    logits  =W[dx,dy,:]
    loss = F.cross_entropy(logits,dzenc)-smoothing_factor*(W**2).mean()

    #backward pass
    W.grad = None
    loss.backward()

    #update 
    W.data+=W.grad*step_size*-1

    print(loss)

tensor(2.2770, grad_fn=<SubBackward0>)
tensor(2.2770, grad_fn=<SubBackward0>)
tensor(2.2770, grad_fn=<SubBackward0>)
tensor(2.2770, grad_fn=<SubBackward0>)
tensor(2.2770, grad_fn=<SubBackward0>)
tensor(2.2770, grad_fn=<SubBackward0>)
tensor(2.2769, grad_fn=<SubBackward0>)
tensor(2.2769, grad_fn=<SubBackward0>)
tensor(2.2769, grad_fn=<SubBackward0>)
tensor(2.2769, grad_fn=<SubBackward0>)


Evaluating the loss over dev data

In [277]:
d_logits = W[dx,dy]
loss = F.cross_entropy(d_logits,dzenc)
loss

tensor(2.2786, grad_fn=<DivBackward1>)

Sampling from dev data

In [383]:
#sampling
g = torch.Generator().manual_seed(2147483647)

for i in range(10):
    out = []
    ix,iy= 0,27 # ix = . iy = <s>
    while True:
        d_logits = W[ix,iy] # gives the 3rd dimension of the weight corresponding to each 27 neurons of the pair ix,iy
        d_counts = d_logits.exp()
        d_p = d_counts/d_counts.sum()

        iz = torch.multinomial(d_p,num_samples=1,replacement=True,generator=g).item()
        if iz==0:
            break
        out.append(itos[iz])
        ix,iy = iy,iz
    print(''.join(out))

junidedianaqad
paufby
adin
kai
ritoper
sathem
dahnaaurinileviassdbduinrwibba
sejyiely
arte
faveumtsyfoltumj
